In [1]:
import numpy as np
import pandas as pd
from dobbel import dobbellogger
dob = dobbellogger()

In [23]:
dob.connect()
dob.log(5, 100, 4, 250)
dob.download()
data = dob.datadf

Logging data for 5s
Done! The data is located in self.datadf


In [24]:
def expq(vector, multiplier=1):
    x, y, z = vector[0] * multiplier, vector[1] * multiplier, vector[2]  * multiplier
    absq = (x ** 2 + y ** 2 + z ** 2) ** 1/2
    w = np.cos(absq)
    sinabs = np.sin(absq)
    xq = x * sinabs / absq
    yq = y * sinabs / absq
    zq = z * sinabs / absq
    return np.array([w, xq, yq, zq])

In [25]:
def quat_mul(p, q):
    p0 = p[0]
    q0 = q[0]
    pv = p[1:]
    qv = q[1:]
    out0 = p0 * q0 - np.dot(pv, qv)
    outv = p0 * qv + q0 * pv + np.cross(pv, qv)
    out = np.array([out0, *outv])
    return out

In [26]:
def quaternion_conjugate(q):
    w, x, y, z = q
    return np.array([w, -x, -y, -z])

In [27]:
# Actual Rotation Fuction - Input = Acceleration vector = 1x3 , rotation_quaternion = 1*4
def rotate_vector(vector, rotation_quaternion):
    # Convert the vector to a quaternion
    vector_quaternion = np.concatenate(([0], vector))

    # Calculate the rotated quaternion
    rotated_quaternion = quat_mul(rotation_quaternion, quat_mul(vector_quaternion, quaternion_conjugate(rotation_quaternion)))

    # Extract the rotated vector from the quaternion
    rotated_vector = rotated_quaternion[1:]

    return np.array(rotated_vector)

In [28]:
def quat_to_euler(q):
    w, x, y, z = q[0], q[1], q[2], q[3]
    psi = np.arctan((2*x*y-2*w*z)/((2*w**2)+(2*x**2)-1))
    theta = -np.arcsin(2*x*z+2*w*y)
    phi = np.arctan((2*y*z-2*w*x)/((2*w**2)+(2*z**2)-1))
    return np.array([psi, theta, phi])

In [29]:
def remove_nan(df):
    counter = 0
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter += 1
    for i in range(counter - 1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    counter = len(df) - 1
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter -= 1
    maxlen = len(df)
    for i in range(maxlen - 1, counter + 1, -1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    for i in range(len(df)):
        df['timestamp'][i] = df['timestamp'][i] - df['timestamp'][0]
    for i in range(len(df)):
        for column in df.columns:
            if np.isnan(df[column][i]):
                deler = (df['timestamp'][i] - df['timestamp'][i - 1]) / (df['timestamp'][i + 1] - df['timestamp'][i - 1])
                df[column][i] = df[column][i - 1] + deler * (df[column][i + 1] - df[column][i - 1])
    return df

In [30]:
def left_quat_mul(q):
    q0, q1, q2, q3 = q
    lqm = np.array([
        [q0, -q1, -q2, -q3],
        [q1, 1, -q3, -q2],
        [q2, q3, 1, -q1],
        [q3, -q2, q1, 1]
    ])
    return lqm

def right_quat_mul(q):
    q0, q1, q2, q3 = q
    lqm = np.array([
        [q0, -q1, -q2, -q3],
        [q1, 1, q3, q2],
        [q2, -q3, 1, q1],
        [q3, q2, -q1, 1]
    ])
    return lqm

In [31]:
angle = np.deg2rad(-39)
rotate_y = np.array([[np.cos(angle),0,np.sin(angle)], [0,1,0],[-np.sin(angle),0,np.cos(angle)]])
trace = np.trace(rotate_y)
q_0 = np.sqrt(1+trace)/2
q_2 = 1/(4*q_0) * (np.sin(angle)+np.sin(angle))
q_rotate = np.array([q_0,0,q_2,0])

In [32]:
data = remove_nan(dob.datadf)

for i, row in data.iterrows():
    data.loc[i, 'x_acc'], data.loc[i, 'y_acc'], data.loc[i, 'z_acc'] = rotate_vector(np.array([row['x_acc'], row['y_acc'], row['z_acc']]), q_rotate)
    data.loc[i, 'x_gyro'], data.loc[i, 'y_gyro'], data.loc[i, 'z_gyro'] = rotate_vector(np.array([row['x_gyro'], row['y_gyro'], row['z_gyro']]), q_rotate)

y11 = np.array([data['x_acc'][0], data['y_acc'][0], data['z_acc'][0]])
gb = y11 / np.linalg.norm(y11)
gn = np.array([0, 0, 1])
mn = np.array([1, 0, 0])
mb = np.array([1, 0, 0])
A = - np.matmul(left_quat_mul(np.array([0, *gn])), right_quat_mul(np.array([0, *gb]))) - np.matmul(left_quat_mul(np.array([0, *mn])), right_quat_mul(np.array([0, *mb])))



# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(A)

# Eigenvalues are sorted in ascending order by np.linalg.eigh, 
# and the corresponding eigenvectors are columns of 'eigenvectors'
max_eigenvalue_index = np.argmax(eigenvalues)
max_eigenvector = eigenvectors[:, max_eigenvalue_index]

print(max_eigenvector)

[ 0.8390625  -0.52837435 -0.07162189 -0.10800454]


In [33]:
for column in data.columns:
    print(column, max(abs(data[column])))

timestamp 4868.0
x_acc 1.2626017124429865
y_acc 1.124755859375
z_acc 0.3117599137278178
x_gyro 157.8958027476944
y_gyro 92.45426940917969
z_gyro 351.1621365624709


In [34]:
data.head(1000)

,timestamp,x_acc,y_acc,z_acc,x_gyro,y_gyro,z_gyro
0,0.0,0.988161,-0.021118,-0.036856,-0.290600,0.800305,0.225635
1,10.0,0.998625,-0.022461,-0.036394,-0.164760,0.289634,0.082348
2,20.0,0.995639,-0.021362,-0.037555,-0.240637,0.381098,0.030711
3,31.0,0.996023,-0.020142,-0.038030,-0.286607,0.495427,-0.045745
4,41.0,0.994626,-0.019531,-0.037275,-0.104367,0.518293,-0.064900
...,...,...,...,...,...,...,...
495,4827.0,-0.004222,0.973389,-0.046771,-0.197467,0.434451,-0.022599
496,4837.0,-0.004506,0.970703,-0.047002,-0.113381,0.373476,-0.150660
497,4847.0,-0.004421,0.971802,-0.045362,-0.105784,0.503049,-0.026816
498,4857.0,-0.004656,0.970947,-0.044295,-0.198594,0.495427,-0.033319


In [35]:
qtmin1tmin1 = max_eigenvector

dqde = np.array([
    [0,0,0],
    [1,0,0],
    [0,1,0],
    [0,0,1]
])

sigma_etha = np.array([
    [np.pi / 9, 0, 0],
    [0, np.pi / 9, 0],
    [0, 0, np.pi / 9]
])

Ptmin1tmin1 = 0.25 * np.matmul(np.matmul(np.matmul(np.matmul(right_quat_mul(qtmin1tmin1), dqde), sigma_etha), dqde.T), right_quat_mul(qtmin1tmin1))

In [36]:
gyro_bias = np.array([-0.1370101, 0.49430553, -0.05923779])
acc_bias = np.array([0.00295479, 0.02747813, -0.04788544])

qtmin1tmin1 = np.reshape(qtmin1tmin1, 4)

v = np.array([0, 0, 0])
p = np.array([0, 0, 0])
q_list = [qtmin1tmin1]
p_list = [p]
v_list = [v]

# std van gyroscope
Q = np.array([
    [0.0152682, 0, 0],
    [0, 0.063694, 0],
    [0, 0, 0.11032866]
])

# std van accelerometer
R = np.array([
    [0.00206182, 0, 0],
    [0, 0.00153152, 0],
    [0, 0, 0.00226686],
])

for i in range(1, len(data['timestamp'])):
    dt = (data['timestamp'][i] - data['timestamp'][i - 1]) / 1000
    gyro_measurement = np.array([data['x_gyro'][i], data['y_gyro'][i], data['z_gyro'][i]])
    omega = gyro_measurement - gyro_bias
    expq_omega = expq(omega, 0.5 * dt * np.pi / 180)
    
    qttmin1 = quat_mul(qtmin1tmin1, expq_omega)

    qtt = qttmin1
    qtmin1tmin1 = np.reshape(qtt, 4)
    
    q_list.append(qtmin1tmin1)
    

In [37]:
for q, timestamp in zip(q_list, data['timestamp']):
    print(timestamp, quat_to_euler(q) * 180 / np.pi)

0.0 [14.88834274  0.34703326 64.44421875]
10.0 [14.8869317   0.34893998 64.44501277]
20.0 [14.88604009  0.34976784 64.44630511]
31.0 [14.88590136  0.34933318 64.4478923 ]
41.0 [14.88595568  0.34918521 64.4475152 ]
50.0 [14.8861685   0.3487073  64.44740465]
60.0 [14.8853824   0.34929493 64.44752387]
70.0 [14.88487438  0.34902712 64.44789438]
80.0 [14.88487647  0.34933076 64.44819194]
91.0 [14.88486112  0.35001545 64.448179  ]
101.0 [14.88535645  0.35069373 64.44795723]
111.0 [14.885953    0.35047958 64.44812582]
120.0 [14.88648989  0.35028684 64.44827754]
121.0 [14.88654955  0.35026543 64.4482944 ]
130.0 [14.8862092   0.3508006  64.44864503]
141.0 [14.88639902  0.35090804 64.44819284]
151.0 [14.88628699  0.35067699 64.44783162]
161.0 [14.88618632  0.35091545 64.44748429]
171.0 [14.88583533  0.35151345 64.4481579 ]
182.0 [14.88558983  0.3517425  64.44855369]
192.0 [14.88515636  0.35174111 64.44821855]
201.0 [14.8846798   0.35209872 64.44790199]
211.0 [14.88380638  0.3528016  64.44669741]